In [ ]:
import zipfile

# Replace 'your_zip_file.zip' with the path to your zip file
zip_file_path = 'stylegan3-tick90.zip'

# Replace 'your_destination_folder' with the path where you want to extract the files
destination_folder = 'your_destination_folder'

# Open the zip file in read mode
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the files to the destination folder
    zip_ref.extractall(destination_folder)


In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib


In [ ]:
def compress_folder(service, folder_id):
    files = []
    query = f"'{folder_id}' in parents"
    results = service.files().list(q=query, fields="nextPageToken, files(id, name, mimeType)").execute()
    items = results.get("files", [])

    with tempfile.TemporaryDirectory() as tmpdirname:
        for item in items:
            print(f"Processing file: {item['name']} (ID: {item['id']})")
            if item['mimeType'] != 'application/vnd.google-apps.folder':
                file_path = os.path.join(tmpdirname, item['name'])
                download_file(service, item['id'], file_path)
                files.append(file_path)

        if not files:
            print("No files found in the folder.")
            return None

        compressed_file = os.path.join(tmpdirname, 'compressed_folder.zip')
        with zipfile.ZipFile(compressed_file, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for file_path in files:
                zipf.write(file_path, os.path.basename(file_path))

        file_metadata = {"name": "compressed_folder.zip", "parents": [folder_id]}
        media = MediaFileUpload(compressed_file, mimetype="application/zip")
        compressed_file = service.files().create(body=file_metadata, media_body=media, fields="id").execute()

    return compressed_file




In [ ]:
import os
import pickle
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload

# Set up the Google Drive API client
def get_drive_service():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', ['https://www.googleapis.com/auth/drive'])
            creds = flow.run_local_server(port=0)

        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    return build('drive', 'v3', credentials=creds)

# Function to download a file from Google Drive
def download_file(service, file_id, output_file):
    try:
        request = service.files().get_media(fileId=file_id)
        with open(output_file, 'wb') as f:
            downloader = MediaIoBaseDownload(f, request)
            done = False
            while done is False:
                status, done = downloader.next_chunk()
                print(f"Downloaded {int(status.progress() * 100)}.")
    except HttpError as error:
        print(f"An error occurred: {error}")

# Download a file from Google Drive
folder_id = '1gMWHYa1YB3GFz2LLOx0Bb_LGf5VhiSYK'  # Replace with your folder ID
output_file = 'folder.zip'  # Replace with the desired output file name

service = get_drive_service()
compressed_file = compress_folder(service, folder_id)
download_file(service, compressed_file['id'], output_file)



NameError: name 'tempfile' is not defined

In [ ]:
import shutil
import os

def zip_folder(folder_path, zip_file_path):
    # Ensure the folder exists
    if not os.path.exists(folder_path):
        raise ValueError(f"Folder '{folder_path}' does not exist")

    # Create a zip file from the folder
    shutil.make_archive(zip_file_path, 'zip', folder_path)

# Usage example
folder_path = 'your_destination_folder'
zip_file_path = 'finished'

try:
    zip_folder(folder_path, zip_file_path)
    print(f"Successfully zipped '{folder_path}' to '{zip_file_path}.zip'")
except Exception as e:
    print(f"Error: {e}")


KeyboardInterrupt: 

In [ ]:
import os
from PIL import Image

input_folder = "resized/necklace"
output_folder = "rgb_necklace"

os.makedirs(output_folder, exist_ok=True)

for file in os.listdir(input_folder):
    if file.lower().endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(input_folder, file)
        img = Image.open(img_path).convert('RGB')
        output_path = os.path.join(output_folder, file)
        img.save(output_path)
